In [ ]:

import os
import logging
from src.config import FOLDER_PATHS, PARAMS
from src.data_loader import load_data
from src.fitting import mpl_adam_fit_I, initialize_params, generate_init_params_I,generate_init_params,initialize_params_I, mpl_adam_fit
from src.evaluation import evaluate_mpl
from src.optimization import optimize_lr_schedule
from tqdm.notebook import tqdm   
import matplotlib.pyplot as plt
import numpy as np
from src.lr_schedules import cosine_lrs, e11_lrs, two_stage_lrs, wsd_lrs, wsdld_lrs


logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s: %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)




def ema_smooth(loss_list, alpha=0.1):
    """
    Manual EMA smoothing (without pandas).
    
    Args:
        loss_list (list or np.array): List of loss values.
        alpha (float): Smoothing factor (0 < alpha ≤ 1).
    
    Returns:
        np.array: Smoothed loss values.
    """
    smoothed = np.zeros_like(loss_list)
    smoothed[0] = loss_list[0]   
    for i in range(1, len(loss_list)):
        smoothed[i] = alpha * loss_list[i] + (1 - alpha) * smoothed[i-1]
    return smoothed



In [ ]:
data = {}
folder_path='/home/zsc/MultiPowerLaw-main'
TRAIN_SET = [
    "100M_20B_cosine.csv",
]

TEST_SET = [
    "100M_20B_wsd.csv",
    "100M_20B_811.csv",
    #"100M_20B_cosine.csv",
]


FILES = TRAIN_SET + TEST_SET
for file_name in TRAIN_SET:
   
    file_path = os.path.join(folder_path, file_name)
    file_data = np.genfromtxt(file_path, delimiter=',', skip_header=1)
    
    Sample = file_data[:, 0].astype(int)
    mask = (Sample % 100) == 99  
    
    
    filtered_steps = Sample[mask]
    filtered_loss = file_data[:, 2].astype(float)[mask]
    
    data[file_name] = {
        "step": filtered_steps+2000-1,
        "loss": filtered_loss,
    }
    
    if "cosine" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = cosine_lrs(2000, total, 1e-3, 1e-4, False)
    elif "wsd" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = wsd_lrs(2000, total, 2000+27126, 1e-3, 1e-4, False)
    elif "811" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = e11_lrs(2000,total,1e-3)

for file_name in TEST_SET:
    
    file_path = os.path.join(folder_path, file_name)
    file_data = np.genfromtxt(file_path, delimiter=',', skip_header=1)
    
    
    loss_list=ema_smooth(file_data[:, 2], alpha=0.2)
    data[file_name] = {
        "step": file_data[:, 0].astype(int)+2000-1,
        "loss": loss_list,
    }
    
    if "cosine" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = cosine_lrs(2000, total, 1e-3, 1e-4, False)
    elif "wsd" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = wsd_lrs(2000, total, 2000+27125, 1e-3, 1e-4, False)
    elif "811" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = e11_lrs(2000,total,1e-3)

In [ ]:
logger.info("Initializing parameters...")
init_param = initialize_params(data,  TRAIN_SET)
init_params = generate_init_params(init_param)
use_I=True
fig_folder='/home/zsc/MultiPowerLaw-main/Results/cos_train'

best_params, best_loss = mpl_adam_fit(
    data, TRAIN_SET, TEST_SET, 
    init_params, fig_folder
)


INFO: Initializing parameters...
INFO: Starting parameter initialization
Initializing Parameters:   1%|▏         | 2/135 [00:00<00:09, 13.72it/s]/home/zsc/MultiPowerLaw-main/src/fitting.py:116: RuntimeWarning: invalid value encountered in log
  r = np.log(loss) - np.log(pred)
Initializing Parameters:   8%|▊         | 11/135 [00:00<00:08, 14.59it/s]/home/zsc/MultiPowerLaw-main/src/fitting.py:116: RuntimeWarning: divide by zero encountered in log
  r = np.log(loss) - np.log(pred)
/home/zsc/miniconda3/envs/llm/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
Initializing Parameters: 100%|██████████| 135/135 [00:09<00:00, 14.94it/s]
INFO: Initialization completed. Best Loss: 0.004127012080696165, Best Params: [  2.70925531   4.41976295   2.08230831 252.52517914]
INFO: Starting MPL fitting with AdamW
INFO: Initializing with parameters: (np.float64(2.709255313292115), np.float64(4.419762949202256), np.float

In [ ]:

logger.info("Evaluating model...")
evaluate_mpl(data, TRAIN_SET, best_params, fig_folder, use_I=False)
evaluate_mpl(data, TEST_SET, best_params, fig_folder, use_I=False)
logger.info(f"Best Loss Achieved: {best_loss:.4f}")

INFO: Evaluating model...
INFO: 100M_20B_cosine.csv
INFO: huber_loss: 0.004139441978285172
INFO: mse_loss: 0.006040329433061562
INFO: rmse_loss: 0.07771955631024641
INFO: mae_loss: 0.03854315477250262
INFO: prede: 0.012684858056851402
INFO: worste: 0.15259080964506402
INFO: r2_score: 0.9701133057275328
INFO: Average Huber_loss: 0.004139441978285172
INFO: Average Mse_loss: 0.006040329433061562
INFO: Average Rmse_loss: 0.07771955631024641
INFO: Average Mae_loss: 0.03854315477250262
INFO: Average Prede: 0.012684858056851402
INFO: Average Worste: 0.15259080964506402
INFO: Average R2_score: 0.9701133057275328
INFO: --------------------------------------------------
INFO: 100M_20B_wsd.csv
INFO: huber_loss: 0.6727815347219516
INFO: mse_loss: 0.023090296866010475
INFO: rmse_loss: 0.15195491721563495
INFO: mae_loss: 0.06394747563791645
INFO: prede: 0.02046179804566009
INFO: worste: 0.3068807598559522
INFO: r2_score: 0.9163474663018016
INFO: 100M_20B_811.csv
INFO: huber_loss: 0.5676987885426189


In [5]:
best_params

[2.635989280714434,
 4.377125677699614,
 2.043470752223466,
 243.2532154282851,
 2.297460356645381,
 0.6062881005472137,
 0.6721041825703479]